In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import itertools as it
import numpy as np
import pickle
import sys
import torch
sys.path.insert(0, '/content/drive/MyDrive/MMA')
sys.path.insert(0, '/content/drive/MyDrive/MMA/MultiBench')

In [ ]:
!pip install memory-profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=5554d2be9851ed5c6797c969c70f34e1703c4328f5719ed2dfa1e39a3d9c939f
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [ ]:
!pip install lime

     |████████████████████████████████| 275 kB 5.1 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=1e95b9d23277aa0ccfdb27f5e998cf1c9ca68c12a541e8813954c14e53c8b825
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [ ]:
from unimodals.common_models import MLP, GRU # noqa
from datasets.mimic.get_data import get_dataloader # noqa
from fusions.common_fusions import Concat # noqa
from training_structures.Supervised_Learning import train, test # noqa

import lime

In [ ]:
# Data sanity check

fp = '/content/drive/MyDrive/MMA/im.pk'
with open(fp, 'rb') as f:
    data = pickle.load(f)
print(data.keys())

dict_keys(['temp', 'ep_tdata', 'adm_features_all', 'adm_labels_all', 'y_icd9'])


In [ ]:
traindata, validdata, testdata = get_dataloader(7, imputed_path=fp, train_shuffle=False)

# build encoders, head and fusion layer
# encoders = [MLP(5, 10, 10, dropout=False).cuda(), GRU(
#     12, 30, dropout=False, batch_first=True).cuda()]
# head = MLP(730, 40, 2, dropout=False).cuda()
# fusion = Concat().cuda()

# train
# train(encoders, fusion, head, traindata, validdata, 20, auprc=True)
# !cp best.pt /content/drive/MyDrive/MMA/saved_models/mimic_baseline.pt

model = torch.load('/content/drive/MyDrive/MMA/saved_models/mimic_baseline.pt').cuda()

100%|██████████| 11/11 [00:06<00:00,  1.63it/s]


In [ ]:
train_X = None
train_y = None

for j in traindata:
    if train_X != None:
        train_X = torch.cat([train_X, torch.cat([j[0], j[1].reshape(j[1].shape[0], -1)], dim=1)], dim=0)
        train_y = torch.cat([train_y, j[2]], dim=0)
    else: 
        train_X = torch.cat([j[0], j[1].reshape(j[1].shape[0], -1)], dim=1)
        train_y = j[2]

train_X = train_X.float().numpy()
train_y = train_y.float().numpy()

In [ ]:
namelist = ['age', 'acquired immunodeficiency syndrome', 'hematologic malignancy', 'metastatic cancer',
'admission type', 'glasgow coma scale', 'systolic blood pressure', 'heart rate', 'body temperature',
'pao2 / fio2 ratio', 'urine output', 'serum urea nitrogen level', 'white blood cells count',
'serum bicarbonate level', 'sodium level', 'potassium level', 'bilirubin level']

static_names = namelist[:5]
timeseries_names = namelist[5:]
idx_to_name = {}

cnt = 0
for i in range(len(static_names)):
    for j in range(len(timeseries_names)):
        for k in range(24):
            idx_to_name[cnt] = f'{static_names[i]}_{timeseries_names[j]}_{k}'
            cnt += 1

In [ ]:
def clf_fn(X):
    X = torch.from_numpy(X).cuda().float()
    static = X[:, :5]
    timeseries = X[:, 5:].reshape(X.shape[0], 24, 12)
    logits = model([static, timeseries])
    return torch.softmax(logits, 1).cpu().detach().numpy()

In [ ]:
import collections
import copy
from functools import partial
import json
import warnings

import numpy as np
import scipy as sp
import sklearn
import sklearn.preprocessing
from sklearn.utils import check_random_state
# from pyDOE2 import lhs
from scipy.stats.distributions import norm

from lime.discretize import QuartileDiscretizer
from lime.discretize import DecileDiscretizer
from lime.discretize import EntropyDiscretizer
from lime.discretize import BaseDiscretizer
from lime.discretize import StatsDiscretizer
from lime import explanation
from lime import lime_base

In [ ]:
class LimeTabularExplainer(object):
    """Explains predictions on tabular (i.e. matrix) data.
    For numerical features, perturb them by sampling from a Normal(0,1) and
    doing the inverse operation of mean-centering and scaling, according to the
    means and stds in the training data. For categorical features, perturb by
    sampling according to the training distribution, and making a binary
    feature that is 1 when the value is the same as the instance being
    explained."""

    def __init__(self,
                 training_data,
                 mode="classification",
                 training_labels=None,
                 feature_names=None,
                 categorical_features=None,
                 categorical_names=None,
                 kernel_width=None,
                 kernel=None,
                 verbose=False,
                 class_names=None,
                 feature_selection='auto',
                 discretize_continuous=True,
                 discretizer='quartile',
                 sample_around_instance=False,
                 random_state=None,
                 training_data_stats=None):
        """Init function.
        Args:
            training_data: numpy 2d array
            mode: "classification" or "regression"
            training_labels: labels for training data. Not required, but may be
                used by discretizer.
            feature_names: list of names (strings) corresponding to the columns
                in the training data.
            categorical_features: list of indices (ints) corresponding to the
                categorical columns. Everything else will be considered
                continuous. Values in these columns MUST be integers.
            categorical_names: map from int to list of names, where
                categorical_names[x][y] represents the name of the yth value of
                column x.
            kernel_width: kernel width for the exponential kernel.
                If None, defaults to sqrt (number of columns) * 0.75
            kernel: similarity kernel that takes euclidean distances and kernel
                width as input and outputs weights in (0,1). If None, defaults to
                an exponential kernel.
            verbose: if true, print local prediction values from linear model
            class_names: list of class names, ordered according to whatever the
                classifier is using. If not present, class names will be '0',
                '1', ...
            feature_selection: feature selection method. can be
                'forward_selection', 'lasso_path', 'none' or 'auto'.
                See function 'explain_instance_with_data' in lime_base.py for
                details on what each of the options does.
            discretize_continuous: if True, all non-categorical features will
                be discretized into quartiles.
            discretizer: only matters if discretize_continuous is True
                and data is not sparse. Options are 'quartile', 'decile',
                'entropy' or a BaseDiscretizer instance.
            sample_around_instance: if True, will sample continuous features
                in perturbed samples from a normal centered at the instance
                being explained. Otherwise, the normal is centered on the mean
                of the feature data.
            random_state: an integer or numpy.RandomState that will be used to
                generate random numbers. If None, the random state will be
                initialized using the internal numpy seed.
            training_data_stats: a dict object having the details of training data
                statistics. If None, training data information will be used, only matters
                if discretize_continuous is True. Must have the following keys:
                means", "mins", "maxs", "stds", "feature_values",
                "feature_frequencies"
        """
        self.random_state = check_random_state(random_state)
        self.mode = mode
        self.categorical_names = categorical_names or {}
        self.sample_around_instance = sample_around_instance
        self.training_data_stats = training_data_stats

        # Check and raise proper error in stats are supplied in non-descritized path
        if self.training_data_stats:
            self.validate_training_data_stats(self.training_data_stats)

        if categorical_features is None:
            categorical_features = []
        if feature_names is None:
            feature_names = [str(i) for i in range(training_data.shape[1])]

        self.categorical_features = list(categorical_features)
        self.feature_names = list(feature_names)

        self.discretizer = None
        if discretize_continuous and not sp.sparse.issparse(training_data):
            # Set the discretizer if training data stats are provided
            if self.training_data_stats:
                discretizer = StatsDiscretizer(
                    training_data, self.categorical_features,
                    self.feature_names, labels=training_labels,
                    data_stats=self.training_data_stats,
                    random_state=self.random_state)

            if discretizer == 'quartile':
                self.discretizer = QuartileDiscretizer(
                        training_data, self.categorical_features,
                        self.feature_names, labels=training_labels,
                        random_state=self.random_state)
            elif discretizer == 'decile':
                self.discretizer = DecileDiscretizer(
                        training_data, self.categorical_features,
                        self.feature_names, labels=training_labels,
                        random_state=self.random_state)
            elif discretizer == 'entropy':
                self.discretizer = EntropyDiscretizer(
                        training_data, self.categorical_features,
                        self.feature_names, labels=training_labels,
                        random_state=self.random_state)
            elif isinstance(discretizer, BaseDiscretizer):
                self.discretizer = discretizer
            else:
                raise ValueError('''Discretizer must be 'quartile',''' +
                                 ''' 'decile', 'entropy' or a''' +
                                 ''' BaseDiscretizer instance''')
            self.categorical_features = list(range(training_data.shape[1]))
            self.pair_features = list(range(5*288))

            # Get the discretized_training_data when the stats are not provided
            if(self.training_data_stats is None):
                discretized_training_data = self.discretizer.discretize(
                    training_data)

        if kernel_width is None:
            kernel_width = np.sqrt(training_data.shape[1]) * .75
        kernel_width = float(kernel_width)

        if kernel is None:
            def kernel(d, kernel_width):
                return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

        kernel_fn = partial(kernel, kernel_width=kernel_width)

        self.feature_selection = feature_selection
        self.base = lime_base.LimeBase(kernel_fn, verbose, random_state=self.random_state)
        self.class_names = class_names

        # Though set has no role to play if training data stats are provided
        self.scaler = sklearn.preprocessing.StandardScaler(with_mean=False)
        self.scaler.fit(training_data)
        self.feature_values = {}
        self.feature_frequencies = {}

        for feature in self.categorical_features:
            if training_data_stats is None:
                if self.discretizer is not None:
                    column = discretized_training_data[:, feature]
                else:
                    column = training_data[:, feature]

                feature_count = collections.Counter(column)
                values, frequencies = map(list, zip(*(sorted(feature_count.items()))))
            else:
                values = training_data_stats["feature_values"][feature]
                frequencies = training_data_stats["feature_frequencies"][feature]

            self.feature_values[feature] = values
            self.feature_frequencies[feature] = (np.array(frequencies) /
                                                 float(sum(frequencies)))
            self.scaler.mean_[feature] = 0
            self.scaler.scale_[feature] = 1

    @staticmethod
    def convert_and_round(values):
        return ['%.2f' % v for v in values]

    @staticmethod
    def validate_training_data_stats(training_data_stats):
        """
            Method to validate the structure of training data stats
        """
        stat_keys = list(training_data_stats.keys())
        valid_stat_keys = ["means", "mins", "maxs", "stds", "feature_values", "feature_frequencies"]
        missing_keys = list(set(valid_stat_keys) - set(stat_keys))
        if len(missing_keys) > 0:
            raise Exception("Missing keys in training_data_stats. Details: %s" % (missing_keys))

    def explain_instance(self,
                         data_row,
                         predict_fn,
                         labels=(1,),
                         top_labels=None,
                         num_features=10,
                         num_samples=5000,
                         distance_metric='euclidean',
                         model_regressor=None,
                         sampling_method='gaussian'):
        """Generates explanations for a prediction.
        First, we generate neighborhood data by randomly perturbing features
        from the instance (see __data_inverse). We then learn locally weighted
        linear models on this neighborhood data to explain each of the classes
        in an interpretable way (see lime_base.py).
        Args:
            data_row: 1d numpy array or scipy.sparse matrix, corresponding to a row
            predict_fn: prediction function. For classifiers, this should be a
                function that takes a numpy array and outputs prediction
                probabilities. For regressors, this takes a numpy array and
                returns the predictions. For ScikitClassifiers, this is
                `classifier.predict_proba()`. For ScikitRegressors, this
                is `regressor.predict()`. The prediction function needs to work
                on multiple feature vectors (the vectors randomly perturbed
                from the data_row).
            labels: iterable with labels to be explained.
            top_labels: if not None, ignore labels and produce explanations for
                the K labels with highest prediction probabilities, where K is
                this parameter.
            num_features: maximum number of features present in explanation
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for weights.
            model_regressor: sklearn regressor to use in explanation. Defaults
                to Ridge regression in LimeBase. Must have model_regressor.coef_
                and 'sample_weight' as a parameter to model_regressor.fit()
            sampling_method: Method to sample synthetic data. Defaults to Gaussian
                sampling. Can also use Latin Hypercube Sampling.
        Returns:
            An Explanation object (see explanation.py) with the corresponding
            explanations.
        """
        if sp.sparse.issparse(data_row) and not sp.sparse.isspmatrix_csr(data_row):
            # Preventative code: if sparse, convert to csr format if not in csr format already
            data_row = data_row.tocsr()
        data, inverse = self.__data_inverse(data_row, num_samples, sampling_method)
        # if sp.sparse.issparse(data):
        #     # Note in sparse case we don't subtract mean since data would become dense
        #     scaled_data = data.multiply(self.scaler.scale_)
        #     # Multiplying with csr matrix can return a coo sparse matrix
        #     if not sp.sparse.isspmatrix_csr(scaled_data):
        #         scaled_data = scaled_data.tocsr()
        # else:
        #     scaled_data = (data - self.scaler.mean_) / self.scaler.scale_
        scaled_data = data.copy()
        distances = sklearn.metrics.pairwise_distances(
                scaled_data,
                scaled_data[0].reshape(1, -1),
                metric=distance_metric
        ).ravel()

        yss = predict_fn(inverse)

        # for classification, the model needs to provide a list of tuples - classes
        # along with prediction probabilities
        if self.mode == "classification":
            if len(yss.shape) == 1:
                raise NotImplementedError("LIME does not currently support "
                                          "classifier models without probability "
                                          "scores. If this conflicts with your "
                                          "use case, please let us know: "
                                          "https://github.com/datascienceinc/lime/issues/16")
            elif len(yss.shape) == 2:
                if self.class_names is None:
                    self.class_names = [str(x) for x in range(yss[0].shape[0])]
                else:
                    self.class_names = list(self.class_names)
                if not np.allclose(yss.sum(axis=1), 1.0):
                    warnings.warn("""
                    Prediction probabilties do not sum to 1, and
                    thus does not constitute a probability space.
                    Check that you classifier outputs probabilities
                    (Not log probabilities, or actual class predictions).
                    """)
            else:
                raise ValueError("Your model outputs "
                                 "arrays with {} dimensions".format(len(yss.shape)))

        # for regression, the output should be a one-dimensional array of predictions
        else:
            try:
                if len(yss.shape) != 1 and len(yss[0].shape) == 1:
                    yss = np.array([v[0] for v in yss])
                assert isinstance(yss, np.ndarray) and len(yss.shape) == 1
            except AssertionError:
                raise ValueError("Your model needs to output single-dimensional \
                    numpyarrays, not arrays of {} dimensions".format(yss.shape))

            predicted_value = yss[0]
            min_y = min(yss)
            max_y = max(yss)

            # add a dimension to be compatible with downstream machinery
            yss = yss[:, np.newaxis]

        feature_names = copy.deepcopy(self.feature_names)
        if feature_names is None:
            feature_names = [str(x) for x in range(data_row.shape[0])]

        if sp.sparse.issparse(data_row):
            values = self.convert_and_round(data_row.data)
            feature_indexes = data_row.indices
        else:
            values = self.convert_and_round(data_row)
            feature_indexes = None

        for i in self.categorical_features:
            if self.discretizer is not None and i in self.discretizer.lambdas:
                continue
            name = int(data_row[i])
            if i in self.categorical_names:
                name = self.categorical_names[i][name]
            feature_names[i] = '%s=%s' % (feature_names[i], name)
            values[i] = 'True'
        categorical_features = self.categorical_features

        discretized_feature_names = None
        if self.discretizer is not None:
            categorical_features = range(data.shape[1])
            discretized_instance = self.discretizer.discretize(data_row)
            discretized_feature_names = copy.deepcopy(feature_names)
            for f in self.discretizer.names:
                discretized_feature_names[f] = self.discretizer.names[f][int(
                        discretized_instance[f])]

        # domain_mapper = TableDomainMapper(feature_names,
        #                                   values,
        #                                   scaled_data[0],
        #                                   categorical_features=categorical_features,
        #                                   discretized_feature_names=discretized_feature_names,
        #                                   feature_indexes=feature_indexes)
        # ret_exp = explanation.Explanation(domain_mapper,
        #                                   mode=self.mode,
        #                                   class_names=self.class_names)
        # if self.mode == "classification":
        #     ret_exp.predict_proba = yss[0]
        #     if top_labels:
        #         labels = np.argsort(yss[0])[-top_labels:]
        #         ret_exp.top_labels = list(labels)
        #         ret_exp.top_labels.reverse()
        # else:
        #     ret_exp.predicted_value = predicted_value
        #     ret_exp.min_value = min_y
        #     ret_exp.max_value = max_y
        #     labels = [0]

        ret_exp = {'intercept': {},
                   'local_exp': {},
                   'score': {},
                   'local_pred': {}}
        
        for label in labels:
            (ret_exp['intercept'][label],
             ret_exp['local_exp'][label],
             ret_exp['score'][label],
             ret_exp['local_pred'][label]) = self.base.explain_instance_with_data(
                    scaled_data,
                    yss,
                    distances,
                    label,
                    num_features,
                    model_regressor=model_regressor,
                    feature_selection=self.feature_selection)

        # if self.mode == "regression":
        #     ret_exp.intercept[1] = ret_exp.intercept[0]
        #     ret_exp.local_exp[1] = [x for x in ret_exp.local_exp[0]]
        #     ret_exp.local_exp[0] = [(i, -1 * j) for i, j in ret_exp.local_exp[1]]

        return ret_exp

    def data_to_pair_data(self, masks):
        static = masks[:, :5]
        timeseries = masks[:, 5:]
        pair_masks = np.asarray([i*j for i in static.T for j in timeseries.T]).T
        return pair_masks

    def __data_inverse(self,
                       data_row,
                       num_samples,
                       sampling_method):
        """Generates a neighborhood around a prediction.
        For numerical features, perturb them by sampling from a Normal(0,1) and
        doing the inverse operation of mean-centering and scaling, according to
        the means and stds in the training data. For categorical features,
        perturb by sampling according to the training distribution, and making
        a binary feature that is 1 when the value is the same as the instance
        being explained.
        Args:
            data_row: 1d numpy array, corresponding to a row
            num_samples: size of the neighborhood to learn the linear model
            sampling_method: 'gaussian' or 'lhs'
        Returns:
            A tuple (data, inverse), where:
                data: dense num_samples * K matrix, where categorical features
                are encoded with either 0 (not equal to the corresponding value
                in data_row) or 1. The first row is the original instance.
                inverse: same as data, except the categorical features are not
                binary, but categorical (as the original data)
        """
        is_sparse = sp.sparse.issparse(data_row)
        if is_sparse:
            num_cols = data_row.shape[1]
            data = sp.sparse.csr_matrix((num_samples, num_cols), dtype=data_row.dtype)
        else:
            num_cols = data_row.shape[0]
            data = np.zeros((num_samples, num_cols))
        categorical_features = range(num_cols)
        if self.discretizer is None:
            instance_sample = data_row
            scale = self.scaler.scale_
            mean = self.scaler.mean_
            if is_sparse:
                # Perturb only the non-zero values
                non_zero_indexes = data_row.nonzero()[1]
                num_cols = len(non_zero_indexes)
                instance_sample = data_row[:, non_zero_indexes]
                scale = scale[non_zero_indexes]
                mean = mean[non_zero_indexes]

            if sampling_method == 'gaussian':
                data = self.random_state.normal(0, 1, num_samples * num_cols
                                                ).reshape(num_samples, num_cols)
                data = np.array(data)
            # elif sampling_method == 'lhs':
            #     data = lhs(num_cols, samples=num_samples
            #                ).reshape(num_samples, num_cols)
            #     means = np.zeros(num_cols)
            #     stdvs = np.array([1]*num_cols)
            #     for i in range(num_cols):
            #         data[:, i] = norm(loc=means[i], scale=stdvs[i]).ppf(data[:, i])
            #     data = np.array(data)
            else:
                warnings.warn('''Invalid input for sampling_method.
                                 Defaulting to Gaussian sampling.''', UserWarning)
                data = self.random_state.normal(0, 1, num_samples * num_cols
                                                ).reshape(num_samples, num_cols)
                data = np.array(data)

            if self.sample_around_instance:
                data = data * scale + instance_sample
            else:
                data = data * scale + mean
            if is_sparse:
                if num_cols == 0:
                    data = sp.sparse.csr_matrix((num_samples,
                                                 data_row.shape[1]),
                                                dtype=data_row.dtype)
                else:
                    indexes = np.tile(non_zero_indexes, num_samples)
                    indptr = np.array(
                        range(0, len(non_zero_indexes) * (num_samples + 1),
                              len(non_zero_indexes)))
                    data_1d_shape = data.shape[0] * data.shape[1]
                    data_1d = data.reshape(data_1d_shape)
                    data = sp.sparse.csr_matrix(
                        (data_1d, indexes, indptr),
                        shape=(num_samples, data_row.shape[1]))
            categorical_features = self.categorical_features
            first_row = data_row
        else:
            first_row = self.discretizer.discretize(data_row)
        data[0] = data_row.copy()
        inverse = data.copy()
        for column in categorical_features:
            values = self.feature_values[column]
            freqs = self.feature_frequencies[column]
            inverse_column = self.random_state.choice(values, size=num_samples,
                                                      replace=True, p=freqs)
            binary_column = (inverse_column == first_row[column]).astype(int)
            binary_column[0] = 1
            inverse_column[0] = data[0, column]
            data[:, column] = binary_column
            inverse[:, column] = inverse_column
        if self.discretizer is not None:
            inverse[1:] = self.discretizer.undiscretize(inverse[1:])
        inverse[0] = data_row

        data = self.data_to_pair_data(data)

        return data, inverse

In [ ]:
explainer = LimeTabularExplainer(train_X.copy(), discretize_continuous=True, random_state=0)

In [ ]:
idx = 733
sample = train_X[idx].copy()
exp = explainer.explain_instance(sample, clf_fn, num_features=5, num_samples=50000)
multimodal_exp = {idx_to_name[k]: v for k, v in exp['local_exp'][1]}

torch.Size([1, 50000, 30])


In [ ]:
multimodal_exp

{'acquired immunodeficiency syndrome_glasgow coma scale_10': 0.027062007232925542,
 'acquired immunodeficiency syndrome_pao2 / fio2 ratio_16': 0.03137376454235384,
 'acquired immunodeficiency syndrome_urine output_4': 0.028831172399166997,
 'admission type_glasgow coma scale_0': 0.043610240876812735,
 'age_glasgow coma scale_0': 0.04628793105763634}

In [ ]:
idx = 962
sample = train_X[idx].copy()
exp = explainer.explain_instance(sample, clf_fn, num_features=5, num_samples=50000)
multimodal_exp = {idx_to_name[k]: v for k, v in exp['local_exp'][1]}

torch.Size([1, 50000, 30])


In [ ]:
multimodal_exp

{'acquired immunodeficiency syndrome_potassium level_16': -0.04884980985662822,
 'acquired immunodeficiency syndrome_sodium level_4': -0.04978007819279831,
 'hematologic malignancy_potassium level_4': -0.04748055134868414,
 'hematologic malignancy_serum bicarbonate level_16': -0.05098543421140426,
 'hematologic malignancy_sodium level_16': -0.061339260293006063}

In [ ]:
exp

{'intercept': {1: 0.7210279753951161},
 'local_exp': {1: [(808, -0.061339260293006063),
   (784, -0.05098543421140426),
   (508, -0.04978007819279831),
   (544, -0.04884980985662822),
   (820, -0.04748055134868414)]},
 'local_pred': {1: array([0.46259284])},
 'score': {1: 0.14020653800964256}}